In [1]:
from IPython.display import Image, display

from react_agent.src.agents.react_agent import ReActAgent

from react_agent.src.agent_tools.sap_help_searcher import SapHelpSearcher
from react_agent.src.agent_tools.troubleshooting_searcher import TroubleshootingSearcher

from react_agent.src.util.llm_proxy import LLMProxy

In [2]:
WITH_ML_FLOW = False

In [3]:
if(WITH_ML_FLOW):
    import mlflow
    # poetry run mlflow server --host 127.0.0.1 --port 8080
    mlflow.set_tracking_uri(uri="http://localhost:8080")
    mlflow.set_experiment("Testing of email ReAct Agent")

    mlflow.langchain.autolog()

In [4]:
tool_list = []

tool_list.append(TroubleshootingSearcher())
tool_list.append(SapHelpSearcher())

In [5]:
agent = ReActAgent(tool_list=tool_list)
print(agent.create_sys_prompt())
#display(Image(agent.get_agent_graph()))


< Role >
You are an expert on Electronic Document Processing and SAP Document and Reporting Compliance.
</ Role >

< Instructions >
Context: Peppol, UBL, eInvoicing.
1. Observation: Start by clearly stating the initial user request, task, or the current state of the problem you need to address.
2. Thought: Analyze the Observation. Break down the task if complex. Identify what information is missing or what specific sub-task needs to be performed next. Formulate a clear reasoning chain: explain why a particular action is needed and how it will help address the Observation or move towards the final goal. Explicitly consider if information validation is required at this stage or after gathering initial data.
3. Action Plan: Based on the Thought, explicitly state: a) The single specific tool you plan to use next. b) The precise input or parameters you will provide to that tool. This step clearly documents the one intended action before execution.
4. Action: Execute the single tool call ex

In [6]:
mapping_question = "Explain how 'Payment Terms' is mapped. Start with 'map_invoice1'."
extensibility_question = "As a Public Cloud customer in Spain, can I extend an existing eDocument customer invoice Process?"
peppol_support = """We as Syntax are currently implementing SAP DRC and are following the official SAP help.
We are currently at the step “Integration of Peppol Exchange with SAP S/4HANA Cloud” and there at the creation of an S/4HANA Public Cloud communication user.
A client certificate should be uploaded for the communication user to be created, as Basic Auth (via user + password) is not supported.

The problem: We do not have this certificate. Where can I get this certificate? What format must it be in?
We are stuck at this step because we are missing the certificate mentioned and it is also not clear to us what kind of certificate is meant here and where to get it.
"""
agent.run_and_print_agent_stream(user_message=peppol_support)

================================ Human Message =================================

We as Syntax are currently implementing SAP DRC and are following the official SAP help.
We are currently at the step “Integration of Peppol Exchange with SAP S/4HANA Cloud” and there at the creation of an S/4HANA Public Cloud communication user.
A client certificate should be uploaded for the communication user to be created, as Basic Auth (via user + password) is not supported.

The problem: We do not have this certificate. Where can I get this certificate? What format must it be in?
We are stuck at this step because we are missing the certificate mentioned and it is also not clear to us what kind of certificate is meant here and where to get it.

================================== Ai Message ==================================

**Observation:** You are implementing SAP Document and Reporting Compliance (DRC) and are currently at the step of integrating Peppol Exchange with SAP S/4HANA Cloud. You need to

In [7]:
LLMProxy().print_usage()

Call Count: 0
Token Usage: {'input_tokens': 0, 'output_tokens': 0, 'total_tokens': 0}
